# Algorithm for datasets aggregation

## Auxiliar function

In [1]:
import pandas as pd

# Auxilar function to open files
def open_df(file_path):
    if ".sav" in file_path:
        df = pd.read_spss(file_path,convert_categoricals=False)
    elif ".xlsx" in file_path:
        df = pd.read_excel(file_path,engine="openpyxl")
    elif ".csv" in file_path:
        df = pd.read_csv(file_path,sep=",")
    print(file_path)
    df.dropna(how='all',inplace=True)
    return df

## Read folder \& and construct metadata dictionary. 

In [2]:
import os
# create dictionary meta-data from aggregation folder

def clean_spaces(l):
    return [w.strip() for w in l]

# Todo: read datasets names from input_file folder

# read folder aggregation
metadata = {}
for dataset in os.listdir('databases'):
    if os.path.isdir(os.path.join('aggregation',dataset)):
        metadata[dataset] = {}
        print(dataset)
        for constructo in os.listdir(os.path.join('aggregation',dataset)):
            if constructo.endswith('.csv'):
                constructo = constructo.replace('.csv','')
                metadata[dataset][constructo] = {}
                
                df = pd.read_csv(os.path.join('aggregation',dataset,constructo+".csv"),sep=',')
                df.to_excel(os.path.join('aggregation',dataset,constructo+".xlsx"),index=False)

                # In linux
                with open(os.path.join('aggregation',dataset,constructo+".csv"),'r',encoding='cp1252') as f:
                # In windows
                #with open(os.path.join('aggregation',dataset,constructo+".csv"),'r',encoding='ansi') as f:
                    lines = f.readlines()
                    print("->",constructo)
                    if constructo == 'socialdemographics':
                        keys = clean_spaces(lines[0].replace('\n','').split(','))
                        values = clean_spaces(lines[1].replace('\n','').split(','))
                        metadata[dataset][constructo] = dict(zip(keys,values))
                    else:
                        keys = clean_spaces(lines[0].replace('\n','').split(','))
                        esp = clean_spaces(lines[1].replace('\n','').split(','))
                        pos = clean_spaces(lines[2].replace('\n','').split(','))
                        metadata[dataset][constructo] = dict(zip(keys[1:],zip(esp[1:],pos[1:])))
                        
for dataset in metadata:
    for constructo in metadata[dataset]:
        print(dataset,constructo,metadata[dataset][constructo])

dataset3.csv
-> socialdemographics
-> variable3
-> variable2
-> variable1
dataset2.csv
-> socialdemographics
-> variable3
-> variable2
-> variable1
dataset1.csv
-> socialdemographics
-> variable3
-> variable2
-> variable1
dataset3.csv socialdemographics {'age': 'Alter', 'sex': 'Geschlecht', 'diagnostic': 'Diagnose'}
dataset3.csv variable3 {'textfeld1': ('pre_feld1_v1', 'post_feld1_v2'), 'textfeld2': ('pre_feld2_v1', 'post_feld2_v2'), 'textfeld3': ('pre_feld3_v1', 'post_feld3_v2'), 'textfeld4': ('pre_feld4_v1', 'post_feld4_v2'), 'textfeld5': ('pre_feld5_v1', 'post_feld5_v2'), 'textfeld6': ('pre_feld6_v1', 'post_feld6_v2'), 'textfeld7': ('pre_feld7_v1', 'post_feld7_v2'), 'textfeld8': ('pre_feld8_v1', 'post_feld8_v2'), 'textfeld9': ('pre_feld9_v1', 'post_feld9_v2'), 'textfeld_total': ('', '')}
dataset3.csv variable2 {'campo_1': ('x_1', 'y_1'), 'campo_2': ('x_2', 'y_2'), 'campo_3': ('x_3', 'y_3'), 'campo_4': ('x_4', 'y_4'), 'campo_total': ('', '')}
dataset3.csv variable1 {'field1': ('E_01'

## Build the algorithms dictionary with columns names to matching between each dataset and the aggregated

In [3]:
def insert_dataset(datset,ma,mar,k,v,i):
    
    ma[dataset][k+'_'+str(i)] = v
    if v!='666' and v !='':
        if v not in mar[dataset]:
            mar[dataset][v] = k+'_'+str(i)
        else:
            count = sum(key.startswith(k+'_') for key in mar[dataset].keys())
            mar[dataset][v+'_'+str(count)] = k+'_'+str(i)
                

df_datasets = {}
map_atributes = {}
map_atributes_rev = {}
for dataset in metadata:
    
    map_atributes[dataset] = {}
    map_atributes_rev[dataset] = {}
    
    df_datasets[dataset]= open_df(os.path.join('databases',dataset))

    for constructo in metadata[dataset]:
        if constructo == 'socialdemographics':
            for key,value in metadata[dataset][constructo].items():
                #if key == 'quototal':
                #     print(dataset,constructo,key,value)
                #print(dataset,key,value)
                values = value.split(';')
                for indx,v in enumerate(values):
                        
                    #map_atributes[dataset][key+'_'+str(indx)] = v
                    #if v!='666' and v !='':
                    #    map_atributes_rev[dataset][v] = key+'_'+str(indx)
                             
                    insert_dataset(dataset,map_atributes,map_atributes_rev,key,v,indx)
        else:
            for key,value in metadata[dataset][constructo].items():

                '''if key == 'qoltotal':
                     print(dataset,constructo,key,value)'''
                
                values = value[0].split(';')
                for indx,v in enumerate(values):

                        #map_atributes[dataset]["pre_"+key+'_'+str(indx)] = v
                        #if v!='666' and v !='':
                        #    map_atributes_rev[dataset][v] = "pre_"+key+'_'+str(indx)
                        insert_dataset(dataset,map_atributes,map_atributes_rev,"pre_"+key,v,indx)
                
                values = value[1].split(';')
                for indx,v in enumerate(values):
                    
                           
                        insert_dataset(dataset,map_atributes,map_atributes_rev,"post_"+key,v,indx)

databases/dataset3.csv
databases/dataset2.csv
databases/dataset1.csv


## Auxiliar functions keeping a logical order in columns

In [4]:
import os

# Determine order of columns
base_folder = 'variables/'

# Impose order of list of variables
#variables_order = ["socialdemographics","Bcis","Bdi","Calgary","Cbq","Cdss","Completion","D15","Dass","Depression","Eq5d","Gaf","Ipsaq","Jtc","Panns","Pbiq","PDD","Pdi","Psyrats","Q-les-q-18","QoLS","Rses","Scip","SECL","Slds","Tmt","Whoqol"]
variables_order = ["socialdemographics","variable1","variable2","variable3"]

count_variables = [2]
variables = ['database','num','mct']
for var in variables_order:
    with open(base_folder+var+".csv","r") as f:
        lines = f.readlines()
        if lines != []:
            aux = lines[0].strip().split(',')
            count_variables.append(len(aux))
            variables.extend(aux)

for var in variables:
    print(var)


def sorting_key(c):

    pre = 0
    if c.startswith('pre_') or c.startswith('post_'):
        
        if c.startswith('pre_'):
            pre = len(variables)
        else:
            pre = 2*len(variables)

        first = c.find("_")
        last = c[::-1].find("_")
        base = c[first+1:-last-1]
        dx = int(c[-last:])
        
    else:
        tokens = c.strip().split('_')
        base = '_'.join(tokens[:-1])
        dx = int(tokens[-1])  

    return [pre,variables.index(base),dx]


# Building dictionary MCT
mct_dict = {}

#base_folder = 'D:\\MyDisk\\Working-PERMEPSY\\databases\\'
base_folder = '../databases'


#with open(os.path.join(base_folder,'input_db','mct_patients.csv'),"r") as f:
#    lines = f.readlines()
#    for line in lines[1:]:
#        dataset,_,condition = line.strip().split(';')
#        mct_dict[dataset] = condition

for dataset in mct_dict:
    print(dataset,mct_dict[dataset])

database
num
mct
age
sex
diagnostic
field1
field2
field3
field4
field5
field6
field_T
campo_1
campo_2
campo_3
campo_4
campo_total
textfeld1
textfeld2
textfeld3
textfeld4
textfeld5
textfeld6
textfeld7
textfeld8
textfeld9
textfeld_total


## Aggregating all datasets

In [5]:
from datetime import datetime

# extrat all columns names
variables_base = ['database_0','num_0']

varia = set()
for dataset in map_atributes_rev:
    varia.update(map_atributes[dataset].keys())
variables_base.extend(list(varia))

variables_base = list(variables_base)#.sort(key=sorting_algorithm)

# create empty dataframe
df = pd.DataFrame(columns=variables_base)

#metadata_aux = ['Group MCT.sav']

#for indx,dataset in enumerate(metadata_aux):
for indx,dataset in enumerate(metadata):

    print(indx,"aggregating",dataset)

    # open dataset and remove spaces in columns names
    df_aux =open_df(os.path.join('databases',dataset))
    
    print("\tTotal pacients",len(df_aux),"in",dataset)
    # remove rows not used in the aggreation: only MCT pacients and empty rows
    cond = '' #mct_dict[dataset].replace("df","df_aux")
    
    #df_aux['mct_0'] = 0
    if cond != '':
        df_aux = df_aux[eval(cond)]


    print("\tTotal net",len(df_aux),"in",dataset)
    df_aux.rename(columns={v:v.strip() for v in df_aux.columns},inplace=True)

    try:
        # extend dataframe with new columns
        not_df_aux_keys = [key for key in map_atributes_rev[dataset].keys() if key not in df_aux.columns]
        
        for col in not_df_aux_keys:
            df_aux[col] = df_aux[col[:-2]]
    
        # remove columns not used in the aggregation
        df_aux = df_aux[map_atributes_rev[dataset].keys()]
    except Exception as error:
        print("WARNING!:",error)
        for c in map_atributes_rev[dataset]:
            if c not in df_aux.columns:
                print("\t",c)

    # add database and num columns
    df_aux['database_0']=dataset
    df_aux['num_0']=df_aux.index

    try:
        
        #rename columns to the base name
        df_aux.rename(columns=map_atributes_rev[dataset],inplace=True)

        # add database and num columns
        #df = pd.concat([df_aux,df],join='outer',axis=0,ignore_index=True)
        df = (df_aux.copy() if df.empty else df.copy() if df_aux.empty else pd.concat([df_aux,df],join='outer',axis=0,ignore_index=True)) # if both DataFrames non empty

    except Exception as error:
        print("ERRROR:",error)

variables_base.sort(key=sorting_key)

today = datetime.today()
# Format as a string
date_string = today.strftime('%Y-%m-%d')

output = 'output'

# Using DataFrame.reindex() to change columns order
df = df.reindex(columns=variables_base)
df.to_csv(output+os.sep+f'permepsy_{date_string}_.csv',index=False)

display(df.head())
print(df.shape)

0 aggregating dataset3.csv
databases/dataset3.csv
	Total pacients 53 in dataset3.csv
	Total net 53 in dataset3.csv
1 aggregating dataset2.csv
databases/dataset2.csv
	Total pacients 51 in dataset2.csv
	Total net 51 in dataset2.csv
2 aggregating dataset1.csv
databases/dataset1.csv
	Total pacients 39 in dataset1.csv
	Total net 39 in dataset1.csv


,database_0,num_0,age_0,sex_0,diagnostic_0,pre_field1_0,pre_field2_0,pre_field3_0,pre_field4_0,pre_field5_0,...,post_textfeld1_0,post_textfeld2_0,post_textfeld3_0,post_textfeld4_0,post_textfeld5_0,post_textfeld6_0,post_textfeld7_0,post_textfeld8_0,post_textfeld9_0,post_textfeld_total_0
0,dataset1.csv,0,39,1,7,0.818327,0.646789,0.568883,0.215947,0.718843,...,529,911,379,930,40,477,743,196,929,NaN
1,dataset1.csv,1,61,0,1,0.714655,0.160662,0.581990,0.714032,0.175524,...,577,914,845,40,768,863,628,734,146,NaN
2,dataset1.csv,2,24,1,4,0.501145,0.257456,0.479422,0.823762,0.475784,...,65,356,182,238,906,599,158,356,122,NaN
3,dataset1.csv,3,36,0,1,0.844773,0.316970,0.533131,0.208293,0.984779,...,610,759,602,766,290,226,453,267,863,NaN
4,dataset1.csv,4,41,1,0,0.148695,0.357471,0.798393,0.082284,0.933044,...,918,385,151,632,344,377,392,991,102,NaN


(143, 49)
